# Naive Bayes Spam Filter From Scratch

Although referred to as "Idiot Bayes" by some, Naive Bayes models perform surprisingly well despite strong independence assumptions in the model. In this project we will build a spam classifier from scratch for SMS messages using Multinomial Naive Bayes which is known to suit situations where data can be turned into counts, such as word counts in text.

You can find more information about the dataset by clicking [here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

## 1. Reading in the data

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
collection = pd.read_csv("SMSSpamCollection", sep="\t", header=None, names=["Label", "SMS"])
collection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
collection["Label"].value_counts(dropna=False)

ham     4825
spam     747
Name: Label, dtype: int64

As we can see, non-spam messages are labelled as "ham" and there's a lot more non-spam mails compared to spam mails.

## 2. Preprocessing

As a first step, we will randomize the entire dataset, then split it into the training and test sets. Training set will consist of 80% of the dataset, leaving 20% for testing purposes.

In [4]:
#Specify random state to make sure the results are reproducable:
sampled_set = collection.sample(frac=1, random_state=1)

#Split into 80% - 20% sets:
train_set = sampled_set[:4459].copy().reset_index(drop=True)
test_set = sampled_set[4459:].copy().reset_index(drop=True)

train_set["Label"].value_counts(normalize=True)

ham     0.865441
spam    0.134559
Name: Label, dtype: float64

In [5]:
test_set["Label"].value_counts(normalize=True)

ham     0.867925
spam    0.132075
Name: Label, dtype: float64

When we first read the dataset, we have checked value counts for the labels. We have found that 87% of the messages were labelled "ham" whereas 13% were labelled spam. We can see that our test and training sets have similar proportions for each value.

We will now work on the SMS column, starting with removing punctuation and making all letters lowercase.

In [6]:
train_set["SMS"] = train_set["SMS"].str.replace(r"\W", " ").str.lower()
train_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


We will now create a vocabulary for the training set. We will use sets for this to prevent duplicates.

In [7]:
train_set["SMS"] = train_set["SMS"].str.split()

vocab = []
for message in train_set["SMS"]:
    for word in message:
        vocab.append(word)
        
vocab = set(vocab) #removing duplicates
vocab = list(vocab)
print(vocab[:5])

['ibm', 'monkey', 'sam', 'perform', 'still']


We will now create a dictionary containing the words as keys and word counts per SMS. Then, we will form a DataFrame from it.

In [8]:
word_counts_per_sms = {word: [0] * len(train_set["SMS"]) for word in vocab}

for index, message in enumerate(train_set["SMS"]):
    for word in message:
        word_counts_per_sms[word][index] += 1
        
word_counts = pd.DataFrame(word_counts_per_sms)
training_set = pd.concat([train_set, word_counts], axis=1)
training_set.head()

,Label,SMS,ibm,monkey,sam,perform,still,secretary,in2,figure,...,rgent,help,alive,fne,ibh,pub,jog,duo,08000930705,unconscious
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 3. Creating our spam filter

We will be using the probabilities of the equations below for classification:

\begin{equation}
P(spam | w_1,w_2, ..., w_n) \propto P(spam) \cdot \prod_{i=1}^{n}P(w_i|spam) \\
P(ham | w_1,w_2, ..., w_n) \propto P(ham) \cdot \prod_{i=1}^{n}P(w_i|ham)
\end{equation}

For each P(w|Spam) and P(w|Ham) in the formulas above, we will be using these equations with Laplace smoothing:

\begin{equation}
P(w_i|spam) = \frac{N_{w_i|spam} + \alpha}{N_{spam} + \alpha \cdot N_{vocabulary}} \\
P(w_i|ham) = \frac{N_{w_i|ham} + \alpha}{N_{ham} + \alpha \cdot N_{vocabulary}}
\end{equation}

In [9]:
n_spam = sum([len(row[1]) for row in training_set if row[0] == "spam"])
n_ham = sum([len(row[1]) for row in training_set if row[0] == "ham"])
n_vocab = len(vocab)

p_spam = training_set["Label"].value_counts()["spam"] / training_set["Label"].shape[0]
p_ham = training_set["Label"].value_counts()["ham"] / training_set["Label"].shape[0]

print(p_spam, p_ham)

0.13455931823278763 0.8654406817672123


Now that we have calculated the constants, we will move on to calculating the parameters.

In [10]:
spam_df = training_set[training_set["Label"] == "spam"].copy()
ham_df = training_set[training_set["Label"] == "ham"].copy()

alpha = 1 #Since we use Laplace smoothing

spam_params = {word: (sum(count) + alpha) / (n_spam + (alpha * n_vocab)) for word, count in spam_df.iloc[:, 2:].iteritems()}
ham_params = {word: (sum(count) + alpha) / (n_ham + (alpha * n_vocab)) for word, count in ham_df.iloc[:, 2:].iteritems()}

We have now calculated our parameters, too. We can now begin writing our function.

## 4. Classification process

In [11]:
def classify(message, verbose=True):

    message = re.sub('\W', ' ', message).lower().split()
    
    try:
        p_spam_given_message = p_spam * np.prod([spam_params[word] for word in message])
        p_ham_given_message = p_ham * np.prod([ham_params[word] for word in message])
    except:
        p_spam_given_message = p_spam
        p_ham_given_message = p_ham
    
    if verbose == True:
        print("P(Spam|message):", p_spam_given_message)
        print("P(Ham|message):", p_ham_given_message)
        
        if p_ham_given_message > p_spam_given_message:
            print("Label: Non-spam")
        elif p_ham_given_message < p_spam_given_message:
            print("Label: Spam")
        else:
            print("This message carries equal probabilities of being non-spam vs. spam. Please consult a human for accurate classification.")
    else:
        if p_ham_given_message > p_spam_given_message:
            return "ham"
        elif p_ham_given_message < p_spam_given_message:
            return "spam"
        else:
            return "needs human classification"

classify("success")

P(Spam|message): 1.728443394127009e-05
P(Ham|message): 0.00022233543526453754
Label: Non-spam


Our function seems to work well!

## 5. Measuring Accuracy

In this step, we will be measuring our function's accuracy on the test set using the formula below:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [12]:
correctly_classified = 0
total_classified = test_set.shape[0]

test_set["prediction"] = test_set["SMS"].apply(classify, verbose=False)

for row in test_set.iterrows():
    row = row[1]
    if row["Label"] == row["prediction"]:
        correctly_classified += 1
        
test_set["prediction"].value_counts()

ham     1062
spam      51
Name: prediction, dtype: int64

In [13]:
accuracy = correctly_classified / total_classified
print("Predictions made by our function are {pred}% accurate.\nCorrect: {correct}\nIncorrect: {inc}\nTotal: {tot}".format(pred=round(accuracy * 100, 2), correct=correctly_classified, inc=total_classified - correctly_classified, tot=total_classified))

Predictions made by our function are 91.37% accurate.
Correct: 1017
Incorrect: 96
Total: 1113


## 6. Conclusion

In this project, we have built a spam classifier from scratch. As we can see in the final step's output, our spam filter has an accuracy of ~91.37% with 1017 correctly classified messages out of 1113.